# Processo infeccioso pulmonar

Neste estudo de caso-controle realizado no Setor de Anatomia e Patologia do Hospital Heliópolis em São Paulo, no período de 1970 a 1982, vamos analisar dados de pacientes com processo infeccioso pulmonar e ajustar um modelo logístico com efeitos principais.

Os dados consistem em 175 pacientes classificados segundo as seguintes variáveis:
- Y: tipo de tumor (1: maligno, 0: benigno)
- IDADE: idade em anos
- SEXO: sexo (0: masculino, 1: feminino)
- HL: intensidade da célula histiócitos-linfócitos (1: ausente, 2: discreta, 3: moderada, 4: intensa)
- FF: intensidade da célula fibrose-frouxa (1: ausente, 2: discreta, 3: moderada, 4: intensa)

Vamos ajustar um Modelo Logístico para investigar a relação entre as variáveis explicativas e a chance de processo infeccioso pulmonar ser maligno.


In [63]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

with open('canc3.dat', 'r') as file:
    content = file.readlines()

dados = []
for line in content:
    if line.strip():
        row = list(map(int, line.split()))
        for i in range(0, len(row), 5):
            group = row[i:i+5]
            dados.append(group)
colunas = ['tipo', 'idade', 'sexo', 'hl', 'ff']
df = pd.DataFrame(dados, columns=colunas)


df['sexo'] = pd.Categorical(df['sexo'])
df['hl'] = pd.Categorical(df['hl'])
df['ff'] = pd.Categorical(df['ff'])

## Fundamentação teórica

O Modelo Logístico é um tipo específico de Modelo Linear Generalizado (GLM). A variável resposta é binária, e a distribuição usada é a distribuição binomial. A escolha da distribuição binomial é justificada porque ela é adequada para modelar variáveis binárias, como a presença ou ausência de um evento.

O Modelo Logístico utiliza a função de ligação logit:

$$
g(\mu) = logit(\mu) = log\left(\frac{\mu}{1 - \mu}\right)
$$

A parte sistemática do modelo logístico é dada por:

$$
\eta = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_p X_p
$$

No caso específico do problema apresentado, temos as seguintes variáveis explicativas: IDADE, SEXO, HL e FF. Portanto, o preditor linear η é dado por:

$$
\eta = \beta_1 + \beta_2 \cdot IDADE + \beta_3 \cdot SEXO + \sum_{i=1}^{4} \beta_{4i} \cdot HL_i+ \sum_{i=1}^{4} \beta_{5i} \cdot FF_i
$$


$\eta$ (eta): $\eta$ é a combinação linear das variáveis independentes (ou preditoras) e seus respectivos coeficientes. Em outras palavras, é a soma ponderada das variáveis, levando em consideração a importância (peso) de cada variável no modelo.

Onde $\beta_1$ é o coeficiente da constante (intercept), $\beta_2$, $\beta_3$, $\beta_{4i}$ e $\beta_{5i}$ são os coeficientes das variáveis IDADE, SEXO, HL e FF, respectivamente.

$Pr\{Y = 1 | \eta\}$: Essa é a probabilidade condicional de Y = 1 (evento de interesse) dado $\eta$. Em outras palavras, é a probabilidade de o evento ocorrer (Y = 1) com base nos valores das variáveis preditoras e seus coeficientes.

${1 + e^{-\eta}}^{-1}$: Essa é a função logística ou sigmóide. A função logística transforma os valores de $\eta$ (que podem variar de -∞ a +∞) em uma probabilidade que varia de 0 a 1. A função é dada por:

$$Pr\{Y = 1 | \eta\} = \frac{1}{1 + e^{-\eta}}$$

Aqui, $e^{-\eta}$ é a exponencial de $-\eta$, que é uma função matemática que cresce rapidamente à medida que $\eta$ aumenta.

Agora, juntando todas as partes:

Calcule $\eta$ com base nos valores das variáveis preditoras e seus coeficientes.
Aplique a função logística a $\eta$ para obter a probabilidade condicional de Y = 1.

Assim, a fórmula apresentada é usada para estimar a probabilidade de o evento de interesse (Y = 1) ocorrer com base nos valores das variáveis preditoras e seus coeficientes em uma regressão logística.

Agora vamos ajustar o modelo logístico aos dados e analisar os resultados.


In [64]:
formula = 'tipo ~ sexo + idade + hl + ff'
logistic_model = smf.glm(formula, data=df, family=sm.families.Binomial())
fit = logistic_model.fit()
print(fit.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   tipo   No. Observations:                  175
Model:                            GLM   Df Residuals:                      166
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -78.701
Date:                Fri, 28 Apr 2023   Deviance:                       157.40
Time:                        08:25:25   Pearson chi2:                     161.
No. Iterations:                     6   Pseudo R-squ. (CS):             0.3631
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8504      1.063     -1.742      0.0

A tabela apresenta os resultados de um modelo de regressão logística generalizado ajustado aos dados sobre processos infecciosos pulmonares. A variável dependente (tipo de tumor) é binária, representada por 1 para maligno e 0 para benigno. As variáveis independentes incluem sexo, idade, intensidade da célula histiócitos-linfócitos (HL) e intensidade da célula fibrose-frouxa (FF).

**Interpretação dos coeficientes:**

Os coeficientes estimados indicam o efeito de cada variável independente na probabilidade logarítmica do tipo de tumor ser maligno, mantendo todas as outras variáveis constantes. Um coeficiente positivo indica um aumento na probabilidade logarítmica, enquanto um coeficiente negativo indica uma diminuição na probabilidade logarítmica.

* Sexo: O coeficiente positivo de 0,784 indica que, mantendo todas as outras variáveis constantes, a probabilidade logarítmica do tipo de tumor ser maligno é maior para indivíduos do sexo feminino (T.2) em comparação com indivíduos do sexo masculino.

* Idade: O coeficiente positivo de 0,065 sugere que, mantendo todas as outras variáveis constantes, um aumento de um ano na idade está associado a um aumento na probabilidade logarítmica do tipo de tumor ser maligno.

* HL: Os coeficientes negativos para HL(2), HL(3) e HL(4) indicam que, mantendo todas as outras variáveis constantes, uma maior intensidade de células histiócitos-linfócitos está associada a uma menor probabilidade logarítmica do tipo de tumor ser maligno.

* FF: Os coeficientes para FF(2), FF(3) e FF(4) são mistos, sugerindo uma relação mais complexa entre a intensidade da célula fibrose-frouxa e a probabilidade logarítmica do tipo de tumor ser maligno.

**Tabela de Estimativas dos Parâmetros:**

| Efeito     | Estimativa     | Efeito   | Estimativa    | Efeito   | Estimativa    |
|------------|----------------|----------|---------------|----------|---------------|
| Constante  | -1,850(1,060)   | HL(2)    | -0,869(0,945)  | FF(2)    | -0,687(0,502)  |
| Sexo       | 0,784(0,469)    | HL(3)    | -2,249(0,968)  | FF(3)    | -1,025(0,525)  |
| Idade      | 0,065(0,013)    | HL(4)    | -3,295(1,466)  | FF(4)    | 0,431(1,123)   |

**Avaliação do ajuste do modelo:**

O desvio do modelo (D(y; µ̂)) foi de 157,40 com 166 graus de liberdade. O desvio do modelo é uma medida de quão bem o modelo ajustado se ajusta aos dados. Um desvio próximo ao número de graus de liberdade (neste caso, 166) indica um
